In [1]:
import torch

In [2]:
import torchaudio

In [3]:
import numpy as np

In [4]:
from scipy.signal import wiener

In [6]:
# Load noisy audio data
noisy_audio_path = '/content/output_audio.wav'
noisy_audio, sr = torchaudio.load(noisy_audio_path)

In [9]:
def spectral_subtraction(audio, alpha=1):
    stft = torch.stft(audio, n_fft=2048, hop_length=512, window=torch.hann_window(2048), return_complex=True)
    magnitude = torch.abs(stft)
    magnitude_sq = magnitude ** 2
    noisy_phase = torch.angle(stft)

    # Estimate noise magnitude
    noise_mag = torch.mean(magnitude_sq, dim=1)
    noise_mag = noise_mag.unsqueeze(1)

    # Spectral subtraction
    estimated_mag = torch.maximum(magnitude_sq - alpha * noise_mag, torch.tensor(0.0))
    estimated_mag = torch.sqrt(estimated_mag)

    # Inverse STFT
    estimated_audio = torch.istft(estimated_mag * torch.exp(1j * noisy_phase),
                                  n_fft=2048, hop_length=512, window=torch.hann_window(2048))
    return estimated_audio


In [10]:
# Denoise the audio using spectral subtraction
denoised_audio = spectral_subtraction(noisy_audio)

In [11]:
# Perform audio enhancement using the Wiener filter
def wiener_filter(audio):
    audio_numpy = audio.numpy()
    denoised_audio = wiener(audio_numpy)
    return torch.from_numpy(denoised_audio)


In [12]:
# Enhance the denoised audio
enhanced_audio = wiener_filter(denoised_audio)

/usr/local/lib/python3.10/dist-packages/scipy/signal/_signaltools.py:1657: RuntimeWarning: divide by zero encountered in divide
  res *= (1 - noise / lVar)
/usr/local/lib/python3.10/dist-packages/scipy/signal/_signaltools.py:1657: RuntimeWarning: invalid value encountered in multiply
  res *= (1 - noise / lVar)


In [13]:
# Save the enhanced audio
torchaudio.save('enhanced_audio.wav', enhanced_audio, sr)